### This is the Chatbot MainFrame, where all interaction happens

In [1]:
# Import functions from funtions.py
from func.functions import NewCustomer, GetCustomer, checkPhone

# Import preprocess functions from preprocess.py
from func.preprocess import preprocess, text_to_vector, get_cosine

# Import Order related functions from orderFunction.py
from func.orderFunction import checkNumber


[nltk_data] Downloading package stopwords to /Users/jr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Import online libraries
import pandas as pd

In [18]:
# Loading all pizza name into one list
pizza_df= pd.read_csv('pizza.csv', sep=';')
listOfPizza = []
for pizzaKind in pizza_df['pizzaName'].unique():
    listOfPizza.append(pizzaKind)

In [22]:
# General Greeting as an opening
import nltk

allPizzaName = pd.read_csv('pizza.csv', sep=';')

initialInput = input('Before we proceed, may I know your phone number?')

# Check whether the phone number is valid or no
if checkPhone(initialInput)[0] == 0:
    print('Please enter the right number')
else:
    isCust = GetCustomer(checkPhone(initialInput)[1])
    if isCust == None:
        print('You are currently not registered in our system. How can I help you today?')
    else:
        orderInput = input(f'Welcome back {isCust[0].capitalize()}! How can I help you today?')
        # User Input
        if checkNumber(preprocess(orderInput)) == False:
            print("Sorry! We only serve one order at a time")
        else:
            # Get Pizza Order
            ## Pizza name is exactly the same with the raw input
            for pizzaName in listOfPizza:
                if pizzaName.lower() in orderInput.lower():
                    print('I gotchu') ## THIS NEED TO BE EDITED FURTHER
                    

I gotchu


In [22]:
test  = 'pepperoni feast'

for i in range(len(pizza_df)):
    if pizza_df.iloc[i, 1] == 'margherita':
        print(pizza_df.iloc[i, 2])

[pizza_df.iloc[i, 2] for i in range(len(pizza_df)) if pizza_df.iloc[i, 1] =='margherita'][0]

'mozzarella Cheese with Tomato Sauce'

In [34]:
class Pizza:
    def __init__(self, name, size, crust, pizza_df=pizza_df):
        self.name = name
        id = [pizza_df.iloc[i, 0] for i in range(len(pizza_df)) if pizza_df.iloc[i, 1] == name][0]
        self.description = pizza_df.iloc[id-1, 2]
        self.calories = pizza_df.iloc[id-1, 3]
        self.serves = pizza_df.iloc[id-1, 4]
        self.size = size
        self.crust = crust
        self.isVegetarian = pizza_df.iloc[id-1, 7]
        self.isVegan = pizza_df.iloc[id-1, 8]
        self.isSpicy = pizza_df.iloc[id-1, 9]
        self.price = pizza_df.iloc[id-1, 10]

    def __str__(self):
        return self.description
    
    def getPrice(self):
        return self.price
        
a = Pizza('supreme', 'small', 'pan')

In [37]:
print(a.getPrice())

18,99
